# Numerical integration by Runge-Kutta-Gill's method

In [84]:
import numpy as np
import pandas as pd
from math import *
from tqdm._tqdm_notebook import tqdm_notebook # シミュレーション進捗率表示
import matplotlib.pyplot as plt  # グラフ描画

## 共通データクラス
---

### 制御状態クラス
* x : 状態変数（x0で初期化）
* dx : 状態変数の微分量
* qx : 数値積分の内部処理で使用

In [85]:
class state:                  # controll state class
  def __init__(self, x0):
    self.dim = len(x0)
    self.x   = x0.copy()      # x
    self.dx  = [0]*len(x0)    # dx
    self.qx  = [0]*len(x0)

### 定数クラス
* 運動方程式、制御器で使用する定数をメンバ変数として設定

In [86]:
class const:                  # constant class
  def __init__(self):
    pass

### 保存用データクラス
 * 時間とデータのセットで格納、データはリストまたはスカラで登録

In [87]:
class logData:
  def __init__(self, x0=None):
    self.reset(x0)
    
  #----------------------------------------------------
  # reset class variables
  #----------------------------------------------------
  def reset(self, x0=None):
    self.tim = []
    self.x = []
    if x0 is not None:
      self.append(0, x0)
      
  #----------------------------------------------------
  # append data
  #----------------------------------------------------
  def append(self, tim, x):
    self.tim.append(tim)    # double
    if isinstance(x, list): # list
      self.x.append(x.copy())
    else:
      self.x.append(x)

## シミュレーションクラス
---
コンストラクタで数値シミュレーションを行う運動方程式と制御器を登録

|No|引数名    |説明|
|:---|:---    |:---|
|1 | func    |運動方程式の関数名を文字列で入力|
|2 | x0      |funcで使用する状態変数の初期値をリストで入力|
|3 | cnst    |funcで使用する定数を構造体またはリストで入力（ない場合はNone）|
|4 | ctrl    |制御器の関数名を文字列で入力（ない場合はNone）|
|5 | period |制御器の制御周期[sec]|
|6 | depends |依存ノード名|

In [134]:
class stepper:               # ODE process class
  #----------------------------------------------------
  # constructor
  #----------------------------------------------------
  def __init__(self, func, x0, cnst=None, ctrl=None, period=0, depends=[]):
    self.reset(x0, cnst)
    self.func    = func      # equation of motion
    self.ctrl    = ctrl      # degital controller
    self.dt_ctrl = period    # period of digital controller
    
    # dependent node name
    if not isinstance(depends, list):
      depends = [depends]
    self.depends = depends
    
  #----------------------------------------------------
  # reset class variables
  #----------------------------------------------------
  def reset(self, x0=None, cnst=None):
    if x0 is None:
      x0 = self.x0    
    # -------
    # model
    self.x0 = x0.copy()
    self.state = state(x0)   # control state
    self.cnst = cnst         # constant
    # -------
    # controller
    self.ctime = 0           # update time
    self.ctrl_in = None      # zero-order input
    # -------
    # result
    self.xlog = logData(x0)  # time, state log
    self.ulog = logData()    # time, input log

  #----------------------------------------------------
  # function handler
  #----------------------------------------------------
  def handler(self, func, *args):
    return func(*args)
  
  #----------------------------------------------------
  # numerical integration by Runge-Kutta-Gill's method
  #----------------------------------------------------
  def step(self, tim, dth_, depends):
    #-------------------------
    # constant
    ra = [0.5, 0.2928932188134525, 1.707106781186548, 0.1666666666666667]
    rb = [2.0, 1.0, 1.0, 2.0]
    rc = [0.5, 0.2928932188134525, 1.707106781186548, 0.5]
    
    #-------------------------  
    # controller
    update_u = False
    if self.ctrl is not None:
      self.ctime -= 1             # count down
      if self.ctime <= 0:
        update_u = True
        self.ctime = ceil(self.dt_ctrl/dth_)
        self.ctrl_in = self.handler(self.ctrl, tim, self.state, self.cnst, depends)

    #-------------------------  
    # motion
    ret = self.handler(self.func, tim, self.state, self.cnst, self.ctrl_in, depends)
    res = True if ret else False
    for rj in range(4):
      for rk in range(self.state.dim):
        r1 = dth_*self.state.dx[rk]
        r2 = ra[rj]*(r1 - rb[rj]*self.state.qx[rk])
        self.state.x[rk]  += r2
        self.state.qx[rk] += 3*r2 - rc[rj]*r1
      if rj == 0 or rj == 2:
        tim += dth_/2.0
      if rj <= 2:
        ret = self.handler(self.func, tim, self.state, self.cnst, self.ctrl_in, depends)
        res = True if ret else res
    
    #-------------------------  
    # logging
    self.xlog.append(tim, self.state.x.copy())
    if update_u:
      self.ulog.append(tim, self.ctrl_in.copy())

    #-------------------------  
    # simulation exit flag
    return res

  #----------------------------------------------------
  # output result as pandas data frame
  #----------------------------------------------------
  def result(self):
    df_x = pd.DataFrame(self.xlog.x, index=self.xlog.tim)
    df_u = None
    if self.ctrl is not None:
      df_u = pd.DataFrame(self.ulog.x, index=self.ulog.tim)
    return(df_x, df_u)

## シミュレータ実行管理クラス
---
* addで追加した複数のstepperクラスをリスト化し、順番に実行
* stepperクラス間のIOを管理

In [168]:
class executor:
  def __init__(self):
    self.nodes = {}    # 登録ノード辞書
    self.order = []    # 実行順リスト

  #-------------------------  
  # 実行ノードの順番表示
  def dispOrder(self):
    print('no, Name, depends')
    print('-----------------')
    for i, nodeName in enumerate(self.order):
      print(i, end=', ')
      depends = self.nodes[nodeName].depends
      if len(depends) > 0:
        print(nodeName, end=''); print('  <---  ' ,end=''); print(depends)
      else:
        print(nodeName)
  #-------------------------  
  # ノードの追加
  def add(self, name, node):
    if isinstance(name, str):
      if isinstance(node, stepper):
        self.nodes[name] = node
        self.order.append(name)
  
  #-------------------------  
  # 依存ノードの状態量を取得
  def dependentState(self, node):
    states = []
    depends = node.depends          # dependent node name
    for depend in depends:
      x = self.nodes[depend].state.x
      states.append(x.copy())
    return states

  #-------------------------  
  # シミュレーションの実行
  def sim(self, dt, smt):
    exit = False
    for tim in tqdm_notebook( np.arange(0, smt, dt) ):
      for nodeName in self.order:
        node = self.nodes[nodeName]     # get processing node      
        s = self.dependentState(node)   # get state of dependent
        ret = node.step(tim, dt, s)     # update time step
        exit = exit | ret               # exit flag
      if exit:
        break

    # get result
    df_x = {}
    df_u = {}
    for nodeName in self.order:
      x , u = self.nodes[nodeName].result()
      df_x[nodeName] = x
      df_u[nodeName] = u
    return(df_x, df_u)

## シミュレーション評価のテスト(サンプル)
---

### 運動方程式（倒立振子）

$\dot x_0 = x_1$ 
$\dot x_1 = a \sin(x_0) + b*u$

* tim : シミュレーション時間
* state : 制御状態量
* const : 定数
* ctrl_in : 外部入力(制御器の出力値が設定される)
* depends : 依存する別ノードの状態リスト

In [169]:
def motion(tim, state, const, ctrl_in, depends):
  #-------------------------  
  x  = state.x
  dx = state.dx
  #-------------------------
  # input
  u = 0
  if ctrl_in is not None:
    u = ctrl_in
    ulim = 0.8                   # input limit
    u = ulim if u>ulim else u    # upper
    u = -ulim if u<-ulim else u  # lower

  # dynamics
  # x[0] = theta
  # x[1] = dtheta
  dx[0] = x[1]
  dx[1] = const.a * sin(x[0]) + const.b * u

### 制御器(SMC制御器)

* tim     : シミュレーション時間
* state   : 制御状態量
* const   : 定数
* depends : 依存する別ノードの状態リスト
* 返値 : 運動方程式への入力値（制御周期毎でゼロ次ホールド入力）

<u>プラント</u>

$\dot x = f(x(t)) + g(x(t))*u(t)$

に対する入力u(t)を演算。

In [170]:
def controller(tim, state, const, depends):
  #-------------------------
  x0 = state.x[0]
  x1 = state.x[1]
  x = np.matrix([x0, x1]).T                  # 状態X
  f = np.matrix([x1, const.a*sin(x0)]).T     # プラント 
  g = np.matrix([0, const.b]).T              # 入力係数
  #-------------------------
  # K>0でリアプノフ関数が負定関数
  # Kを大きくとるとロバスト性が上がる
  K = const.K 
  # tanhのx=0付近での滑らかさ
  # alfaが小さい方が滑らかさが上がり、チャタリングが起きにくい
  alfa = const.alfa 
  #-------------------------
  p = np.matrix([1,2]).T                     # すべり面パラメータ
  S = p.T*x                                  # すべり面（p'x）
  Sx = p.T                                   # Sのx偏微分
  Sxg = np.linalg.inv(Sx*g)                  # (p'B)^(-1)
  u = -Sxg*Sx*f - Sxg*K*tanh(alfa*S)         # 符号関数をtanhで緩和
  return u[0,0]   # 計算結果はスカラであるが、Matrix型のため[0,0]を追加

### （サンプル）シミュレーション実行・結果描画

In [171]:
def test(modelName='test'):
  #----------------------------------------------------
  # シミュレーションで使用する定数を指定
  # 指定した定数は運動方程式及び制御器の引数に渡される
  c = const()
  c.a = 1.0
  c.b = 1.0
  c.alfa = 1
  c.K = 2

  #----------------------------------------------------
  # 時間ステップ更新処理用のstepperを設定
  # stepperクラスでRunge-Kutta-Gill法による4次精度常微分方程式の数値積分を実施
  # stepperクラスでは、運動方程式(func)と制御器（ctrl）に関数ポインタを設定
  s1 = stepper(func    = motion,     # equation of motion
               x0      = [2*pi,0],   # initial state
               cnst    = c,          # constant
               ctrl    = controller, # digital controller
               period  = 0.2,        # control period
               depends = []          # dependent node name
               )

  s2 = stepper(func    = motion,     # equation of motion
               x0      = [2*pi,0],   # initial state
               cnst    = c,          # constant
               ctrl    = controller, # digital controller
               period  = 0.1,        # control period
               depends = 'test'          # dependent node name
               )
  
  #----------------------------------------------------
  # 数値シミュレーション実行用のノードを設定
  # ノード登録順が時間ステップの更新順
  # ノード名前によりノード間の依存がある場合の参照・結果の辞書登録を実施
  exe = executor()
  exe.add('test' , s1)
  exe.add('test2', s2)
  exe.dispOrder()        # ノード実行順・依存関係を表示

  #----------------------------------------------------
  # 数値シミュレーションを実施（刻み時間dt[sec]、終了時間[smt]）
  # シミュレーション結果はpandasのデータフレーム（df_）
  df_x, df_u = exe.sim(dt=0.05, smt=15)
  dfx = df_x[modelName]  # 状態量の時間履歴
  dfu = df_u[modelName]  # 制御入力の時間履歴

  #----------------------------------------------------
  # グラフ描画
  fig = plt.figure(figsize=(15,5))
  N = 2
  #----
  ax = fig.add_subplot(N,1,1)
  ax.plot(dfx.index, dfx.iloc[:,0], label="x"+str(0), color='b', linewidth=2, linestyle='-')
  ax.set_xlabel('time[sec]')
  plt.legend(loc='lower right')
  plt.grid(which='major',color='black',linestyle='--')
  #----
  ax = fig.add_subplot(N,1,2)
  ax.plot(dfx.index, dfx.iloc[:,1], label="x"+str(1), color='b', linewidth=2, linestyle='-')
  ax.set_xlabel('time[sec]')
  plt.legend(loc='lower right')
  plt.grid(which='major',color='black',linestyle='--')
  #----
  fig = plt.figure(figsize=(15,5))
  ax = fig.add_subplot(1,1,1)
  ax.step(dfu.index, dfu.iloc[:,0], label="u"+str(0), color='b', linewidth=2, linestyle='-')
  ax.set_xlabel('time[sec]')
  plt.legend(loc='lower right')
  plt.grid(which='major',color='black',linestyle='--')
  #----
  plt.show()

In [173]:
# インポートして使用する場合は不要なのでコメントアウト
#test('test2')